## Data Information 

alldat contains 39 sessions from 10 mice, data from Steinmetz et al, 2019. Time bins for all measurements are 10ms, starting 500ms before stimulus onset. The mouse had to determine which side has the highest contrast. For each dat = alldat[k], you have the fields below. For extra variables, check out the extra notebook and extra data files (lfp, waveforms and exact spike times, non-binned).

* dat['mouse_name']: mouse name
* dat['date_exp']: when a session was performed
* dat['spks']: neurons by trials by time bins.
* dat['brain_area']: brain area for each neuron recorded.
* dat['ccf']: Allen Institute brain atlas coordinates for each neuron.
* dat['ccf_axes']: axes names for the Allen CCF.
* dat['contrast_right']: contrast level for the right stimulus, which is always contralateral to the recorded brain areas.
* dat['contrast_left']: contrast level for left stimulus.
* dat['gocue']: when the go cue sound was played.
* dat['response_times']: when the response was registered, which has to be after the go cue. The mouse can turn the wheel before the go cue (and nearly always does!), but the stimulus on the screen won't move before the go cue.
* dat['response']: which side the response was (-1, 0, 1). When the right-side stimulus had higher contrast, the correct choice was -1. 0 is a no go response.
* dat['feedback_time']: when feedback was provided.
* dat['feedback_type']: if the feedback was positive (+1, reward) or negative (-1, white noise burst).
* dat['wheel']: turning speed of the wheel that the mice uses to make a response, sampled at 10ms.
* dat['pupil']: pupil area (noisy, because pupil is very small) + pupil horizontal and vertical position.
* dat['face']: average face motion energy from a video camera.
* dat['licks']: lick detections, 0 or 1.
* dat['trough_to_peak']: measures the width of the action potential waveform for each neuron. Widths <=10 samples are "putative fast spiking neurons".
* dat['%X%_passive']: same as above for X = {spks, pupil, wheel, contrast_left, contrast_right} but for passive trials at the end of the recording when the mouse was no longer engaged and stopped making responses.
* dat['prev_reward']: time of the feedback (reward/white noise) on the previous trial in relation to the current stimulus time.
* dat['reaction_time']: ntrials by 2. First column: reaction time computed from the wheel movement as the first sample above 5 ticks/10ms bin. Second column: direction of the wheel movement (0 = no move detected).

## Data Retrieval

In [1]:
import os, requests

fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)


## Data Loading

In [2]:
import numpy as np

alldat = np.array([])
for j in range(len(fname)):
  alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

orig_dat = alldat #Save a copy of original

print(alldat[11].keys())

dict_keys(['spks', 'wheel', 'pupil', 'response', 'response_time', 'bin_size', 'stim_onset', 'contrast_right', 'contrast_left', 'brain_area', 'feedback_time', 'feedback_type', 'gocue', 'mouse_name', 'date_exp', 'trough_to_peak', 'active_trials', 'contrast_left_passive', 'contrast_right_passive', 'spks_passive', 'pupil_passive', 'wheel_passive', 'prev_reward', 'ccf', 'ccf_axes', 'cellid_orig', 'reaction_time', 'face', 'face_passive', 'licks', 'licks_passive'])


## Load Reaction Time, Calculate New Behavioral Variables

In [12]:
import scipy.io as sio
import requests
import io

rtstruct_url = 'https://github.com/ayeshav/nma_project/blob/master/RTStruct.mat?raw=true'
r = requests.get(rtstruct_url)

temp = sio.loadmat(io.BytesIO(r.content))
RTStruct = temp['RTStruct']
rt = RTStruct[0]['allRt']
rt_trial_indices = RTStruct[0]['trialInd']

alldat=orig_dat #Set alldat to original
nSessions = len(alldat)

for iSession in range(nSessions):
  dat = alldat[iSession]
  relative_contrast = dat['contrast_left'] - dat['contrast_right']
  reaction_time = rt[iSession]
  rt_trial_ind = rt_trial_indices[iSession] -1 # Subtract 1 because MATLAB is 1-indexed, Python is 0-indexed

  nTrials = len(dat['gocue'])

  alldat[iSession]['reaction_time'] = np.empty((nTrials))
  alldat[iSession]['reaction_time'][:] = np.nan
  alldat[iSession]['reaction_time'][rt_trial_ind] = reaction_time

  alldat[iSession]['relative_contrast'] = relative_contrast
  alldat[iSession]['difficulty'] = abs(relative_contrast)

  alldat[iSession]['GO_trial'] = (relative_contrast!=0)
  
  alldat[iSession]['GO_correct'] = np.logical_or(np.logical_and(relative_contrast<0,dat['response']<0),np.logical_and(relative_contrast>0,dat['response']>0))
  alldat[iSession]['GO_incorrect'] = np.logical_or(np.logical_and(relative_contrast<0,dat['response']>0),np.logical_and(relative_contrast>0,dat['response']<0))

  alldat[iSession]['GO_correct_right'] = np.logical_and(alldat[iSession]['GO_correct'],relative_contrast<0)
  alldat[iSession]['GO_correct_left'] = np.logical_and(alldat[iSession]['GO_correct'],relative_contrast>0)

  alldat[iSession]['GO_incorrect_right'] = np.logical_and(alldat[iSession]['GO_incorrect'],relative_contrast<0)
  alldat[iSession]['GO_incorrect_left'] = np.logical_and(alldat[iSession]['GO_incorrect'],relative_contrast>0)

